# Azure Data Explorer (ADX)

## Azure Authentication

In [ ]:
# interactive azure aad login
!az login

In [ ]:
# az account show
# az account list
# az account set --subscription "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
from azure.cli.core import get_default_cli
get_default_cli().invoke(["account", "show"])

## Conect to ADX and Run KQL Query

In [ ]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
import pandas as pd

In [ ]:
KUSTO_URI = "https://XXXXXXXXXX.XXXXX.kusto.windows.net"

In [ ]:
kcsb = KustoConnectionStringBuilder.with_az_cli_authentication(KUSTO_URI)
client = KustoClient(kcsb)

In [ ]:
database = "XXXXXXXXXX"
query = "XXXXXXXXX | take 10"
try:
    response = client.execute(database, query)
except KustoServiceError as error:
    print("Error:", error)

In [ ]:
dataframe_from_result_table(result.primary_results[0])

## API Hack to Automatically Generate CSV File Schemas and Mappings

In [ ]:
import json
import requests
import pandas as pd

In [ ]:
# get via deverloper tools in browser, get while in ADX and going to data ingest page
my_auth_token = "Bearer XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
# SAS token for raw data file(s) to ingest in Azure Storage 
my_url = "https://XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:
# new table name
table_name = "my_table_name"

In [ ]:
# payload
my_payload =  {
    "db" : database,
    "csl" : "evaluate external_data_schema('" + my_url +"', 'csv', dynamic({\"useFirstRowAsHeader\":true}))" 
}
my_payload_json = json.dumps(my_payload)

In [ ]:
# adx api request 
with requests.Session() as ses:
    response = ses.post(
        url=f"{KUSTO_URI}/v2/rest/query",
        headers= {
            "authorization" : my_auth_token,
            "content-type" : "application/json; charset=UTF-8",
        },
        data=my_payload_json,
    )
    if not response.ok:
        print(response, response.content, response.status_code)
    else:
        result = json.loads(response.content)
        df = pd.DataFrame(result[2]["Rows"], columns=["name","index","system_type","type"])
        df['name'] = df['name'].str.replace("[", "", regex=True).str.replace("]", "", regex=True).str.replace("'", "", regex=True)
        df['name_safe'] = "['" + df['name'] + "']"
        df['index'] = df['index'].astype('str')

In [ ]:
# generate KQL schemas, mappings, transforms, and policies 
staging_schema = f".create table ['STAGING_{table_name}'] (\n    " + ",\n    ".join(df['name_safe'] + ":string") + "\n    )"
production_schema = f".create table ['{table_name}'] (\n    " + ",\n    ".join(df['name_safe'] + ":" + df['type']) + "\n    )"
mapping = f".create table ['STAGING_{table_name}'] ingestion csv mapping 'STAGING_{table_name}_mapping'\n   ```[\n        " + ",\n        ".join('{"column":"' + df['name'] +'", "Properties":{"Ordinal":"' + df['index'] +'"}}') + "\n   ]```"
transform_function = f".create function {table_name}_transform()\n    " + "{\n" + f"        STAGING_{table_name}\n" + "        ...\n        | project " + ', '.join(df['name']) + "\n    }"
policy_1 = f".alter table {table_name} policy update\n    @'[" + '{"IsEnabled": true, "Source": "STAGING_' + table_name + f', "Query": "{table_name}_transform()' + '", "IsTransactional": true, "PropagateIngestionProperties": false}]' + "'"
policy_2 = f".alter-merge table STAGING_{table_name} policy retention softdelete = 0s"

In [ ]:
# view generated KQL
print(staging_schema, end='\n\n')
print(production_schema, end='\n\n')
print(mapping, end='\n\n')
print(transform_function, end='\n\n')
print(policy_1, end='\n\n')
print(policy_2, end='\n\n')